# 📝 Exercise M6.01

The aim of this notebook is to investigate if we can tune the hyperparameters
of a bagging regressor and evaluate the gain obtained.

We will load the California housing dataset and split it into a training and
a testing set.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(as_frame=True, return_X_y=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0, test_size=0.5)

Create a `BaggingRegressor` and provide a `DecisionTreeRegressor`
to its parameter `base_estimator`. Train the regressor and evaluate its
statistical performance on the testing set using the mean absolute error.

In [5]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

bagging_regressor = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_jobs=2)
bagging_regressor.fit(data_train, target_train)
predictions = bagging_regressor.predict(data_test)
score = mean_absolute_error(target_test, predictions)
print(f"Basic mean absolute error of the bagging regressor:\n"
      f"{mean_absolute_error(target_test, predictions):.2f} k$")

Basic mean absolute error of the bagging regressor:
37.08 k$


Now, create a `RandomizedSearchCV` instance using the previous model and
tune the important parameters of the bagging regressor. Find the best
parameters  and check if you are able to find a set of parameters that
improve the default regressor still using the mean absolute error as a
metric.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">You can list the bagging regressor's parameters using the <tt class="docutils literal">get_params</tt>
method.</p>
</div>

In [7]:
for param in bagging_regressor.get_params().keys():
    print(param)

base_estimator__ccp_alpha
base_estimator__criterion
base_estimator__max_depth
base_estimator__max_features
base_estimator__max_leaf_nodes
base_estimator__min_impurity_decrease
base_estimator__min_impurity_split
base_estimator__min_samples_leaf
base_estimator__min_samples_split
base_estimator__min_weight_fraction_leaf
base_estimator__random_state
base_estimator__splitter
base_estimator
bootstrap
bootstrap_features
max_features
max_samples
n_estimators
n_jobs
oob_score
random_state
verbose
warm_start


In [8]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": randint(10, 30),
    "max_samples": [0.5, 0.8, 1.0],
    "max_features": [0.5, 0.8, 1.0],
    "base_estimator__max_depth": randint(3, 10),
}
search = RandomizedSearchCV(
    bagging_regressor, param_grid, n_iter=20, scoring="neg_mean_absolute_error"
)
_ = search.fit(data_train, target_train)

In [9]:
import pandas as pd

columns = [f"param_{name}" for name in param_grid.keys()]
columns += ["mean_test_score", "std_test_score", "rank_test_score"]
cv_results = pd.DataFrame(search.cv_results_)
cv_results = cv_results[columns].sort_values(by="rank_test_score")
cv_results["mean_test_score"] = -cv_results["mean_test_score"]
cv_results

,param_n_estimators,param_max_samples,param_max_features,param_base_estimator__max_depth,mean_test_score,std_test_score,rank_test_score
6,24,1.0,0.8,9,38.964775,0.187912,1
0,16,0.8,0.8,9,39.536004,0.928086,2
15,18,1.0,1.0,8,40.984119,1.001077,3
16,21,0.8,0.8,8,41.007194,1.445983,4
19,15,0.5,0.8,7,42.853530,1.161107,5
18,24,1.0,0.8,7,42.884094,1.061657,6
9,11,1.0,1.0,7,43.645791,1.109273,7
5,27,0.8,0.5,9,43.835780,1.695072,8
17,10,0.5,0.5,9,44.649131,1.871270,9
8,27,1.0,0.5,9,44.714905,1.620202,10


In [10]:
target_predicted = search.predict(data_test)
print(f"Mean absolute error after tuning of the bagging regressor:\n"
      f"{mean_absolute_error(target_test, target_predicted):.2f} k$")

Mean absolute error after tuning of the bagging regressor:
39.96 k$


We see that the predictor provided by the bagging regressor does not need much hyperparameter tuning compared to a single decision tree. We see that the bagging regressor provides a predictor for which tuning the hyperparameters is not as important as in the case of fitting a single decision tree.